# Part 1 

In [ ]:
with open("/home/bchatillon/Documents/Advent-of-Code/2020/20_data.txt") as f:
    data = f.read().splitlines()

In [ ]:
def get_adj(pos):
    return [tuple(map(sum, zip(pos, vector))) for vector in [(-1, 0), (0, 1), (1, 0), (0, -1)]]

def get_border_representation(pos, image):
    vectors = get_adj(pos)
    tile_repre = []
    for idx, vector in enumerate(vectors):
        face = None
        if vector in image:
            tab = image[vector][0]
            borders = get_border(tab)
            face = borders[(idx + 2) % 4][::-1]
        tile_repre.append(face)
    
    return tile_repre
        
def possible_places(image):
    places = set(image.keys())
    possible_places = set(adj for pos in places for adj in get_adj(pos))
    return possible_places - places

def flip(tab):
    return tab[::-1]

def rotate(tab):
    return list(map(list, zip(*tab[::-1])))

def get_border(tab):
    return [tab[0], list(map(lambda l: l[-1], tab)), tab[-1][::-1], list(map(lambda l: l[0], tab))[::-1]]

def all_rotation(tab):
    yield tab
    for _ in range(3):
        tab = rotate(tab)
        yield tab

def all_possibilities(tab):
    yield from all_rotation(tab)
    tab = flip(tab)
    yield from all_rotation(tab)

In [ ]:
def find_place(tile_id, image, tiles):
    if tile_id in list(map(lambda t: t[1], image.values())):
        return None
    if not image:
        image[(0, 0)] = (tiles[tile_id], tile_id)
        return None
    
    all_borders = [(tab, get_border(tab)) for tab in all_possibilities(tiles[tile_id])]
    
    for possible_place in possible_places(image):
        rerp_borders = get_border_representation(possible_place, image)
        
        for tab, borders in all_borders:
            if all(repr_border is None or repr_border == border for repr_border, border in zip(rerp_borders, borders)):
                image[possible_place] = (tab, tile_id)
                return None
    return tile_id

In [ ]:
import regex as re
import itertools
pattern = re.compile(r"Tile (\d+):")

image = {}
tiles = {}
ids_not_place = []
for b, group in itertools.groupby(data, key=bool):
    if not b:
        continue
    
    group = list(group)
    key = pattern.match(group[0]).group(1)
    tab = [list(line) for line in group[1:]]
    tiles[key] = tab
    
    if (id_not_place := find_place(key, image, tiles)) is not None:
        ids_not_place.append(id_not_place)
    else:
        id_copy = ids_not_place.copy()
        for id_not_place in ids_not_place:
            if find_place(id_not_place, image, tiles) is None:
                id_copy.remove(id_not_place)
        ids_not_place = id_copy

In [ ]:
ids_not_place

In [ ]:
image

In [ ]:
while len(image) != len(tiles):
    id_copy = ids_not_place.copy()
    for id_not_place in ids_not_place:
        if find_place(id_not_place, image, tiles) is None:
            id_copy.remove(id_not_place)
    if len(ids_not_place) == len(id_copy):
        raise ValueError("Can not reduce list")
    ids_not_place = id_copy        


In [ ]:
min_x, max_x = min(image, key=lambda t : t[0])[0], max(image, key=lambda t : t[0])[0]
min_y, max_y = min(image, key=lambda t : t[1])[1], max(image, key=lambda t : t[1])[1]

coords = [(min_x, min_y), (min_x, max_y), (max_x, min_y), (max_x, max_y)]
print(coords)

In [ ]:
import math

math.prod(int(image[coord][1]) for coord in coords)

## Part 2

In [ ]:
import numpy as np
scale = len(image[(0, 0)][0]) - 2 
len_tot_x = scale*(1 + max_x - min_x)
array = np.empty((len_tot_x, scale*(1 + max_y - min_y)), dtype=str)

x_offset = abs(min_x) if min_x < 0 else 0
y_offset = abs(min_y) if min_y < 0 else 0

for coord, (tab, _) in image.items():
    x, y = coord
    x += x_offset
    y += y_offset
    xmax, ymax = (x+1)*scale, (y+1)*scale
    xmin, ymin = x*scale, y*scale
    np_tab = np.array(tab)[1:-1, 1:-1]
    array[xmin:xmax, ymin:ymax] = np_tab

In [ ]:
sum(val == "#" for line in array for val in line)

In [ ]:
monster =["                  # ", "#    ##    ##    ###", " #  #  #  #  #  #   "]
monster_size = len(monster[0])

In [ ]:
for elem in monster:
    print(elem)

In [ ]:
monster_padded = monster.copy()
monster_padded[0] = monster_padded[0].ljust(len_tot_x, ".").lstrip().replace(" ", ".")
monster_padded[1] = monster_padded[1].ljust(len_tot_x, ".").replace(" ", ".")
monster_padded[2] = monster_padded[2].rstrip().replace(" ", ".")

In [ ]:
for elem in monster_padded:
    print(elem)

In [ ]:
flat_monster = "".join(monster_padded).replace(" ", ".")
pattern_monster = re.compile(flat_monster)

In [ ]:
def pos_to_coord(pos):
    return pos // len_tot_x, pos % len_tot_x 

In [ ]:
from copy import deepcopy
for tab in all_possibilities(array):
    tab_copy = deepcopy(tab)
    flat_tab = "".join("".join(l) for l in tab_copy)
    matches =  list(pattern_monster.finditer(flat_tab, overlapped=True))
    for match in matches:
        print(match)
        start, end = match.span()
        
        if pos_to_coord(start + len(monster_padded[0]) + len(monster[1]))[0] != pos_to_coord(start + len(monster_padded[0]))[0]:
            print("multiyline..")
        
        for i, pos in enumerate(range(start, end)):
            if flat_monster[i] == "#":
                x, y = pos_to_coord(pos)
                tab_copy[x][y] = "0"
    print(sum(val == "#" for line in tab_copy for val in line))
    print()
    if matches:
        break